In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
import xgboost as xgb

In [4]:
df = pd.read_csv('../data/train_data_encoded.csv')
#df.price = np.log(df.price)
print('shape:', df.shape)
df.head(5)

shape: (36944, 8)


,quality,color,clarity,price,weight_ES,depth_percent_ES,table_percent_ES,volume_ES
0,3,6,1,6.353,-0.625000,0.357143,0.333333,-0.591767
1,4,5,5,9.183,0.484375,0.571429,-0.333333,0.550637
2,4,4,3,7.983,0.031250,-0.071429,0.666667,0.052269
3,2,3,1,8.371,0.593750,0.928571,0.000000,0.624043
4,3,3,4,6.588,-0.531250,0.285714,0.666667,-0.511469


In [5]:
X = df.drop(["price"], axis = 1)
y = df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size   = 0.8, random_state = 42)

def metricas(y_test, y_train, y_test_pred, y_train_pred, tipo_modelo):
    resultados = {'MAE': [metrics.mean_absolute_error(y_test, y_test_pred), metrics.mean_absolute_error(y_train, y_train_pred)],
                'MSE': [metrics.mean_squared_error(y_test, y_test_pred), metrics.mean_squared_error(y_train, y_train_pred)],
                'RMSE': [np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)), np.sqrt(metrics.mean_squared_error(y_train, y_train_pred))],
                'R2':  [metrics.r2_score(y_test, y_test_pred), metrics.r2_score(y_train, y_train_pred)],
                 "set": ["test", "train"]}
    df = pd.DataFrame(resultados)
    df["modelo"] = tipo_modelo
    return df

In [6]:
params = { 'max_depth': [7],
           'learning_rate': [0.02],
           'n_estimators': [750],
           'colsample_bytree': [0.9]}
xgbr = xgb.XGBRegressor(seed = 20, objective ='reg:squarederror')
clf = GridSearchCV(estimator=xgbr, 
                   param_grid=params,
                   scoring='neg_mean_squared_error', 
                   verbose=1,
                   n_jobs=-1)
clf.fit(X_train, y_train)

print("Best parameters:", clf.best_params_)
print("Lowest RMSE: ", (-clf.best_score_)**(1/2.0))

y_pred_gb_test = clf.predict(X_test)
y_pred_gb_train= clf.predict(X_train)

results_xg_boost = metricas(y_test, y_train, y_pred_gb_test, y_pred_gb_train, "XG Boost GridSearch")
results_xg_boost.style.background_gradient(cmap='coolwarm')

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Best parameters: {'colsample_bytree': 0.9, 'learning_rate': 0.02, 'max_depth': 7, 'n_estimators': 750}
Lowest RMSE:  0.09533163763044003


,MAE,MSE,RMSE,R2,set,modelo
0,0.070501,0.008697,0.093256,0.990997,test,XG Boost GridSearch
1,0.063518,0.007008,0.083713,0.992655,train,XG Boost GridSearch


In [8]:
df_test = pd.read_csv('../data/test_data_encoded.csv')
df_test

,quality,color,clarity,weight_ES,depth_percent_ES,table_percent_ES,volume_ES
0,4,1,2,-0.609375,-1.000000,0.333333,-0.586243
1,3,1,2,0.828125,0.714286,1.000000,0.758368
2,3,6,2,1.484375,0.071429,0.666667,1.465556
3,3,6,1,0.062500,-0.928571,-0.333333,0.098797
4,0,5,1,1.234375,2.071429,-0.666667,1.206485
...,...,...,...,...,...,...,...
13480,3,3,2,0.609375,-1.642857,1.000000,0.619552
13481,2,6,2,0.296875,0.142857,1.000000,0.288656
13482,4,4,3,-0.640625,0.142857,-1.233333,-0.623494
13483,4,1,2,0.843750,-1.642857,0.666667,0.886611


In [9]:
#predict test to submit competition
y_pred_clf_test = clf.predict(df_test)
submission_4 = pd.DataFrame(y_pred_clf_test).reset_index()
print('shape:', y_pred_clf_test.shape)
submission_4.columns = ['id', 'price']
submission_4

shape: (13485,)


,id,price
0,0,6.057106
1,1,8.519402
2,2,9.452552
3,3,7.841348
4,4,9.068610
...,...,...
13480,13480,8.510431
13481,13481,8.269464
13482,13482,6.288179
13483,13483,8.620482


In [10]:
submission_4.to_csv('../output/submission_4.csv', index=False)